# OMIM annotation of constrained genes

In [5]:
import numpy as np
import pandas as pd
import re

In [18]:
gm_columns = [
    "chr",
    "start",
    "end",
    "cl",
    "ccl",
    "mim",
    "symbol",
    "gene_name",
    "approved_symbol",
    "entrez",
    "ensg",
    "comment",
    "phenotype",
    "mouse",
]

gm = pd.read_csv(
    "../data/genemap2.txt",
    sep="\t",
    comment="#",
    header=None,
    names=gm_columns,
    usecols=["ensg", "approved_symbol", "mim", "phenotype"],
)
gm = gm.dropna(subset=["ensg", "phenotype"])

gm["phenotype"] = gm.phenotype.str.split(";")
gm = gm.explode("phenotype")

re_long = r"^(.*),\s(\d{6})\s\((\d)\)(|, (.*))$"
re_short = r"^(.*)\((\d)\)(|, (.*))$"

def match_re(string, _re = re_long):
    string = string.strip()
    return re.match(_re, string)

pheno_long = gm.phenotype.apply(match_re, _re = re_long).dropna()
pheno_short = gm.loc[gm.index.difference(pheno_long.index), "phenotype"].apply(match_re, _re = re_short)

In [19]:
gm.shape

(7420, 4)

In [20]:
pheno_long.shape

(7315,)

In [21]:
pheno_short.shape

(40,)

In [23]:
x = gm.phenotype.apply(match_re, _re = re_short)

In [24]:
x.head()

24    <re.Match object; span=(0, 52), match='Immunod...
25    <re.Match object; span=(0, 103), match='Myasth...
30    <re.Match object; span=(0, 53), match='?Immuno...
32    <re.Match object; span=(0, 83), match='Ehlers-...
32    <re.Match object; span=(0, 118), match='Spondy...
Name: phenotype, dtype: object

0